### A plan how the essay scoring experiment will be done

In [ ]:
import scipy as scipy
import stats as stats

'''
1- using the tfIDF vectroizer or the sbert pretrained model to vectroize the sentences into vectors and get the features vectors.

2- then apped the clusters vectors an array of 19 elements to the end of the vectors array.


3- train the model using of the shallow machine learning model.......

'''

In [47]:
from keras.models import Sequential
from keras import layers
from keras_preprocessing.sequence import pad_sequences
from keras.utils import np_utils
import pandas as pd
from sklearn.svm import SVC
from sklearn import datasets, svm, tree, metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
#from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import sys
import numpy as np
import seaborn as sns
from scipy.sparse import csr_matrix, vstack, hstack
from scipy import sparse
from mpl_toolkits.mplot3d import Axes3D
import nltk
import re
import matplotlib.pyplot as pltb
from sklearn.preprocessing import StandardScaler

### Building the right file names and reading the files

In [52]:
df = pd.read_csv(r"D:\UDE\6th Semester\MEMS\MEWS Data\TRACE_Datensatz_transposed_220308.csv", sep='\t')
y = df['Code1_IN_AR_Gesamteindruck_Argumentation_3_4_0']  # y is the gold standards

listEssayNames = df['DocumentID']

listEssayNames = [i.removeprefix("00") for i in listEssayNames]

listEssayNames = [i.replace("al","aI") for i in listEssayNames]

listRightNames = []
for essay in listEssayNames:
    x = essay.split("X")
    temp = x[0]+"_"+x[1]+".txt"
    listRightNames.append(temp)
    #print(x)

from pathlib import Path

essays =[]
df_fileName = pd.DataFrame({})
for essayRigt in listRightNames:
    my_file = Path(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\ALLEssaysAllPrompts", essayRigt)
    if my_file.is_file():
        text = my_file.read_text(encoding='utf-8-sig')
        essays.append(text)

#df_fileName.to_excel("essaysToFileName.xlsx")

### Accuracy and Adjcent Accuracy

In [53]:
def custom_adjacent_accuracy_score(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sum(np.abs(y_pred - y_true) <= 1) / len(y_pred)

### CountVectorizer Test

In [54]:
essays_train, essays_test, y_train, y_test = train_test_split(essays, y, test_size=0.25, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit_transform(essays_train)
X_train = vectorizer.transform(essays_train)
X_test  = vectorizer.transform(essays_test)

print(X_train.shape)
print(X_test.shape)

classifier = LogisticRegression(max_iter=100)
classifier.fit(X_train, y_train)
score1 = classifier.score(X_test, y_test)
pred = classifier.predict(X_test)
print("Accuracy:", score1)
print("ِAdjecent Accuracy:",custom_adjacent_accuracy_score(y_test,pred))
print(confusion_matrix(y_test, pred))

(1716, 12399)
(573, 12399)
Accuracy: 0.3228621291448517
ِAdjecent Accuracy: 0.7678883071553229
[[ 0  0  3  3  2  0  0]
 [ 0 12 24 23  6  0  0]
 [ 0 22 44 48 30  5  0]
 [ 2 21 71 93 43  4  1]
 [ 0  4 17 39 33  4  1]
 [ 0  1  3  5  4  3  0]
 [ 0  0  0  0  2  0  0]]


C:\Users\Ahmed\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Sbert Test With Clusters as Features

In [ ]:
#essays_train, essays_test, y_train, y_test = train_test_split(essays, y, test_size=0.25, random_state=1000)

model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

X_embeddings = model.encode(essays, show_progress_bar =True, device="cuda")

df_goldStandarsdClusters = pd.read_excel("FinalResults.xlsx")

clusterFeatureVectormulti = []
finalX_Train = []

for i in range (len(essays)):

    clusterVectorFeatureSingle =  [0 for col in range(19)]

    listEssayClusers = df_goldStandarsdClusters[df_goldStandarsdClusters['fileName'].str.contains(listRightNames[i])]['cluster']

    if (len(listEssayClusers) ==0):
        print("error")

    for cluster in listEssayClusers:
        try:
            clusterVectorFeatureSingle[cluster] = 1
        except:
            print(cluster)

    clusterFeatureVectormulti.append(clusterVectorFeatureSingle)

for i in range(len(X_embeddings)):
    arr = np.append(X_embeddings[i],clusterFeatureVectormulti[i])
    finalX_Train.append(arr)

npa = np.asarray(finalX_Train, dtype=np.ndarray)

X_train, X_test, y_train, y_test = train_test_split(npa, y, test_size=0.25, random_state=1000)

print(X_train.shape)
print(X_test.shape)

classifier = SVC(kernel="linear", C=0.025) #LogisticRegression()  #DecisionTreeClassifier(max_depth=4)
classifier.fit(X_train, y_train)
score1 = classifier.score(X_test, y_test)
pred = classifier.predict(X_test)
print("Accuracy:", score1)

print(confusion_matrix(y_test, pred))
print("QWK: ", cohen_kappa_score(y_test,pred,weights="quadratic"))

### Sbert test without clusters as Features

In [95]:
essays_train, essays_test, y_train, y_test = train_test_split(essays, y, test_size=0.25, random_state=1000)

model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

X_train = model.encode(essays_train, show_progress_bar =True, device="cuda")

X_test  = model.encode(essays_test, show_progress_bar =True, device="cuda")

print(X_train.shape)
print(X_test.shape)

classifier = LogisticRegression() #SVC(kernel="linear", C=0.025) #LogisticRegression()  #DecisionTreeClassifier(max_depth=4)
classifier.fit(X_train, y_train)
score1 = classifier.score(X_test, y_test)
pred = classifier.predict(X_test)
print("Accuracy:", score1)
print("ِAdjecent Accuracy:",custom_adjacent_accuracy_score(y_test,pred))

print("pearson correaltion 1 ",scipy.stats.pearsonr(y_test, pred))
print("pearson correaltion 2", stats.pearsonr(y_test, pred))

print(confusion_matrix(y_test, pred))

print("QWK: ", cohen_kappa_score(y_test,pred,weights="quadratic"))

Batches:   0%|          | 0/54 [00:00<?, ?it/s]

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

(1716, 768)
(573, 768)
Accuracy: 0.41012216404886565
ِAdjecent Accuracy: 0.8394415357766143
pearson correaltion 1  (0.2260998151449603, 4.4640303139982e-08)
pearson correaltion 2 (0.2260998151449603, 4.4640303139982e-08)
[[  0   0   4   3   1   0   0]
 [  0   0  19  46   0   0   0]
 [  0   0  34 103  12   0   0]
 [  0   1  31 183  20   0   0]
 [  0   0   9  71  18   0   0]
 [  0   0   0  14   2   0   0]
 [  0   0   0   2   0   0   0]]
QWK:  0.1740535968522976


C:\Users\Ahmed\AppData\Local\Temp\ipykernel_17008\244250591.py:20: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  print("pearson correaltion 2", stats.pearsonr(y_test, pred))


In [97]:
classifier = DecisionTreeClassifier(max_depth=1) #SVC(kernel="linear", C=0.025) #LogisticRegression()  #DecisionTreeClassifier(max_depth=4)
classifier.fit(X_train, y_train)
score1 = classifier.score(X_test, y_test)
pred = classifier.predict(X_test)
print("Accuracy:", score1)
print("ِAdjecent Accuracy:",custom_adjacent_accuracy_score(y_test,pred))
print("pearson correaltion 1 ",scipy.stats.pearsonr(y_test, pred))
print("pearson correaltion 2", stats.pearsonr(y_test, pred))
print(confusion_matrix(y_test, pred))

print("QWK: ", cohen_kappa_score(y_test,pred,weights="quadratic"))


# type(essays_train[0])
# df_essaysNames = pd.read_excel("essaysToFileName.xlsx")
# df_goldStandarsdClusters = pd.read_excel("FinalResults.xlsx")
# # essayName = df_essaysNames.query(f"essay == {essays_train[0]}")['fileName'].iloc[0]
#
# res = df_essaysNames[df_essaysNames['essay'].str.contains(essays_train[0])].iloc[0]['fileName']
#
# print(res)
#
# listEssayClusers  = df_goldStandarsdClusters[df_goldStandarsdClusters['fileName'].str.contains(res)]['FinalCluster']
#
# for inn in listEssayClusers:
#     print(inn)

Accuracy: 0.4223385689354276
ِAdjecent Accuracy: 0.8621291448516579
pearson correaltion 1  (0.22975158069575605, 2.6635531439179146e-08)
pearson correaltion 2 (0.22975158069575605, 2.6635531439179146e-08)
[[  0   0   6   2   0   0   0]
 [  0   0  26  39   0   0   0]
 [  0   0  51  98   0   0   0]
 [  0   0  44 191   0   0   0]
 [  0   0  14  84   0   0   0]
 [  0   0   3  13   0   0   0]
 [  0   0   0   2   0   0   0]]
QWK:  0.16373445248796503


C:\Users\Ahmed\AppData\Local\Temp\ipykernel_17008\2844418081.py:8: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  print("pearson correaltion 2", stats.pearsonr(y_test, pred))


### tfIdf vectorizer with only n-gram baseline

In [88]:
from scipy.stats import stats
import scipy
from sklearn.preprocessing import normalize
from sklearn.metrics import precision_recall_fscore_support as score
x
essays_train, essays_test, y_train, y_test = train_test_split(essays, y, test_size=0.25, random_state=1000)

vectorizer = TfidfVectorizer(min_df=1, stop_words='english')
vectorizer.fit_transform(essays_train)
X_train = vectorizer.transform(essays_train)
X_test  = vectorizer.transform(essays_test)

# X_train = normalize (X_train)
# X_test = normalize (X_test)

print(X_train.shape)
print(X_test.shape)

classifier = SVC(kernel="linear") ##LogisticRegression() #SVC(kernel="linear", C=0.025) #
classifier.fit(X_train, y_train)
score1 = classifier.score(X_test, y_test)
pred = classifier.predict(X_test)

precision, recall, fscore, support = score(y_test, pred)

print("Accuracy:", score1)
print("ِAdjecent Accuracy:",custom_adjacent_accuracy_score(y_test,pred))
# print("recall:", recall, "fscore:", fscore, "support:", support, "precision:", precision)
print(confusion_matrix(y_test, pred))

print("pearson correaltion 1 ",scipy.stats.pearsonr(y_test, pred))
print("pearson correaltion 2", stats.pearsonr(y_test, pred))
#print(np.corrcoef("the value for person correaltaion is: ",np.array(y_test).any(), np.array(pred).any()))
print("weighted kappa: ", cohen_kappa_score(y_test,pred,weights="quadratic"))

(1716, 12109)
(573, 12109)
Accuracy: 0.40663176265270506
ِAdjecent Accuracy: 0.8481675392670157
[[  0   0   4   4   0   0   0]
 [  0   0  20  43   2   0   0]
 [  0   0  40 103   6   0   0]
 [  0   0  39 187   9   0   0]
 [  0   0  11  81   6   0   0]
 [  0   0   0  15   1   0   0]
 [  0   0   0   1   1   0   0]]
pearson correaltion 1  (0.2024922409462375, 1.022411070788281e-06)
pearson correaltion 2 (0.2024922409462375, 1.022411070788281e-06)
weighted kappa:  0.14987146409625973


C:\Users\Ahmed\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_17008\1195003522.py:32: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  print("pearson correaltion 2", stats.pearsonr(y_test, pred))


### tfIdf vectorizer with Clusters as Features

In [ ]:
vectorizer = TfidfVectorizer(min_df=1, stop_words='english')
vectorizer.fit_transform(essays)
X_embeddings = vectorizer.transform(essays)

df_goldStandarsdClusters = pd.read_excel("FinalResults.xlsx")

clusterFeatureVectormulti = []
finalX_Train = []

for i in range (len(essays)):

    clusterVectorFeatureSingle =  [0 for col in range(19)]

    listEssayClusers = df_goldStandarsdClusters[df_goldStandarsdClusters['fileName'].str.contains(listRightNames[i])]['cluster']

    if (len(listEssayClusers) ==0):
        print("error")

    for cluster in listEssayClusers:
        clusterVectorFeatureSingle[cluster] = clusterVectorFeatureSingle[cluster] + 1

    clusterFeatureVectormulti.append(clusterVectorFeatureSingle)

Xa = X_embeddings[0]
Xb = sparse.csr_matrix(clusterFeatureVectormulti[0])
diff_n_rows = Xa.shape[0] - Xb.shape[0]
Xb_new = vstack((Xb, csr_matrix((diff_n_rows, Xb.shape[1]))))
X_final = hstack((Xa, Xb_new))

for i in range(1,len(essays)):
    Xa = X_embeddings[i]
    Xb = sparse.csr_matrix(clusterFeatureVectormulti[i])
    diff_n_rows = Xa.shape[0] - Xb.shape[0]
    Xb_new = vstack((Xb, csr_matrix((diff_n_rows, Xb.shape[1]))))
    X_semiFinal = hstack((Xa, Xb_new))
    X_final = vstack((X_final, X_semiFinal))

X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.25, random_state=1000)

print(X_train.shape)
print(X_test.shape)

classifier = LogisticRegression() #LogisticRegression()  #DecisionTreeClassifier(max_depth=4)
classifier.fit(X_train, y_train)
score1 = classifier.score(X_test, y_test)
pred = classifier.predict(X_test)
print("Accuracy:", score1)

print(confusion_matrix(y_test, pred))

print(cohen_kappa_score(y_test,pred,weights="quadratic"))

### Testing

In [29]:
# listFinal = []
# Xa = X_embeddings[0]
# Xb = sparse.csr_matrix(clusterFeatureVectormulti[0])
#
#
# diff_n_rows = Xa.shape[0] - Xb.shape[0]
#
# Xb_new = vstack((Xb, csr_matrix((diff_n_rows, Xb.shape[1]))))
# #where diff_n_rows is the difference of the number of rows between Xa and Xb
#
# X_final = hstack((Xa, Xb_new))
#
#
# Xa = X_embeddings[1]
# Xb = sparse.csr_matrix(clusterFeatureVectormulti[1])
#
#
# diff_n_rows = Xa.shape[0] - Xb.shape[0]
#
# Xb_new = vstack((Xb, csr_matrix((diff_n_rows, Xb.shape[1]))))
# #where diff_n_rows is the difference of the number of rows between Xa and Xb
#
# X_final1 = hstack((Xa, Xb_new))


# t = sparse.csr_matrix(clusterFeatureVectormulti[0])
# t
# X_embeddings[0]

# X_final
# X_final = vstack((X_final, X_final1))


# an_array = np.array([1, 2])
# another_array = np.array([3, 4])
# stacked_array = np.vstack((an_array, another_array))

In [30]:
# text= ['hello my name is ahmed and I am a data scientist','data science is very important field','data science is not that hard']
# text1 = ['you are watching unfold data science']
# vectorizer = CountVectorizer()
#
# v = vectorizer.fit(text)
# xv = vectorizer.fit_transform(text)
# print(xv.toarray())
# print(xv.shape)
# vectorizer.get_feature_names_out()
# cv = TfidfVectorizer(min_df=1, stop_words='english')
# xv = cv.fit_transform(text)
# cv.get_feature_names_out()
# print(xv.toarray())

In [ ]:
# from sklearn.preprocessing import normalize
#
# norm = normalize (X_train_WithClusers)
# norm
# scaler = StandardScaler()
# scaler.fit(X_train_WithClusers)
# X_TrainedScaled = scaler.fit_transform(X_train_WithClusers)
#X_TrainedScaled[0]# from sklearn.preprocessing import normalize
#
# norm = normalize (X_train_WithClusers)
# norm
# scaler = StandardScaler()
# scaler.fit(X_train_WithClusers)
# X_TrainedScaled = scaler.fit_transform(X_train_WithClusers)
#X_TrainedScaled[0]

# Quadratically weighted Kabba - Ready functions

In [33]:
# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def Cmatrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = Cmatrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

### QWK between the first and the second rater

In [6]:
y1QWK = df['Code1_IN_AR_Gesamteindruck_Argumentation_3_4_0']
y2QWK = df['Code2_IN_AR_Gesamteindruck_Argumentation_3_4_0']

# cohen_kappa_score(y_test,pred,weights="quadratic")

cohen_kappa_score(y1QWK,y2QWK,weights="quadratic")

0.3977300242869025

### Fuctions to append the clusters to the feature vectors

In [89]:
def readTextFromFileName(listNames):
    returnList= []
    for i in range (len(listNames)):
        my_file = Path(r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\ALLEssaysAllPrompts", listNames[i])
        if my_file.is_file():
            text = my_file.read_text(encoding='utf-8-sig')
            returnList.append(text)
    return returnList

def AppendClustersToTfIDf(essaysNamesList , tfIDFVectorsArray, vectorizer):

    df_goldStandarsdClusters = pd.read_excel("FinalResults.xlsx")
    clusterFeatureVectormulti = []

    for i in range (len(essaysNamesList)):
        clusterVectorFeatureSingle =  [0 for col in range(19)]
        listEssayClusers = df_goldStandarsdClusters[df_goldStandarsdClusters['fileName'].str.contains(essaysNamesList[i])]['cluster']

        if (len(listEssayClusers) ==0):
            print("error")

        for cluster in listEssayClusers:
            clusterVectorFeatureSingle[cluster] = clusterVectorFeatureSingle[cluster] + 1

        clusterFeatureVectormulti.append(clusterVectorFeatureSingle)

    if (vectorizer == "tfidf"):
        print("tfIDf")
        ### append logic ############
        Xa = tfIDFVectorsArray[0]
        Xb = sparse.csr_matrix(clusterFeatureVectormulti[0])
        diff_n_rows = Xa.shape[0] - Xb.shape[0]
        Xb_new = vstack((Xb, csr_matrix((diff_n_rows, Xb.shape[1]))))
        X_final = hstack((Xa, Xb_new))

        for i in range(1,len(essaysNamesList)):
            Xa = tfIDFVectorsArray[i]
            Xb = sparse.csr_matrix(clusterFeatureVectormulti[i])
            diff_n_rows = Xa.shape[0] - Xb.shape[0]
            Xb_new = vstack((Xb, csr_matrix((diff_n_rows, Xb.shape[1]))))
            X_semiFinal = hstack((Xa, Xb_new))
            X_final = vstack((X_final, X_semiFinal))
        return X_final
    else:
        print("sbert")
        finalX_Train = []
        for i in range(len(tfIDFVectorsArray)):
            arr = np.append(tfIDFVectorsArray[i],clusterFeatureVectormulti[i])
            finalX_Train.append(arr)
        npa = np.asarray(finalX_Train, dtype=np.ndarray)
        return npa


### TFIDF with features Clusters - Increasing the feature vector instead of binary values

In [90]:
essaysNames_train, essaysNames_test, y_train, y_test = train_test_split(listRightNames, y, test_size=0.25, random_state=1000)

essays_train=readTextFromFileName(essaysNames_train)
essays_test =readTextFromFileName(essaysNames_test)


vectorizer = TfidfVectorizer(min_df=1, stop_words='english')
vectorizer.fit_transform(essays_train)
X_train = vectorizer.transform(essays_train)
X_test  = vectorizer.transform(essays_test)
print(X_train.shape)
print(X_test.shape)

X_train_WithClusers = AppendClustersToTfIDf(essaysNames_train,X_train,"tfidf")
X_test_WithClusters = AppendClustersToTfIDf(essaysNames_test,X_test,"tfidf")
print(X_train_WithClusers.shape)
print(X_test_WithClusters.shape)

### classification logic #######################

classifier = LogisticRegression() #LogisticRegression()  #DecisionTreeClassifier(max_depth=4)
classifier.fit(X_train_WithClusers, y_train)
score1 = classifier.score(X_test_WithClusters, y_test)
pred = classifier.predict(X_test_WithClusters)
print("Accuracy:", score1)
print("ِAdjecent Accuracy:",custom_adjacent_accuracy_score(y_test,pred))
print("pearson correaltion 1 ",scipy.stats.pearsonr(y_test, pred))
print("pearson correaltion 2", stats.pearsonr(y_test, pred))
print(confusion_matrix(y_test, pred))

print(cohen_kappa_score(y_test,pred,weights="quadratic"))

(1716, 12109)
(573, 12109)
tfIDf
tfIDf
(1716, 12128)
(573, 12128)
Accuracy: 0.37521815008726006
ِAdjecent Accuracy: 0.8516579406631762
pearson correaltion 1  (0.26751836587620426, 7.579791244585139e-11)
pearson correaltion 2 (0.26751836587620426, 7.579791244585139e-11)
[[  1   0   4   3   0   0   0]
 [  1   3  26  32   3   0   0]
 [  1   5  49  84  10   0   0]
 [  0   3  50 151  31   0   0]
 [  0   1  12  75  10   0   0]
 [  0   0   0  14   1   1   0]
 [  0   0   0   2   0   0   0]]
0.2417616892911011


C:\Users\Ahmed\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Ahmed\AppData\Local\Temp\ipykernel_17008\4202545438.py:28: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  print("pearson correaltion 2", stats.pearsonr(y_test, pred))


In [93]:
classifier = SVC(kernel="linear") #LogisticRegression()  #DecisionTreeClassifier(max_depth=4)
classifier.fit(X_train_WithClusers, y_train)
score1 = classifier.score(X_test_WithClusters, y_test)
pred = classifier.predict(X_test_WithClusters)
print("Accuracy:", score1)
print("ِAdjecent Accuracy:",custom_adjacent_accuracy_score(y_test,pred))

print("pearson correaltion 1 ",scipy.stats.pearsonr(y_test, pred))
print("pearson correaltion 2", stats.pearsonr(y_test, pred))
print(confusion_matrix(y_test, pred))

print(cohen_kappa_score(y_test,pred,weights="quadratic"))

Accuracy: 0.3856893542757417
ِAdjecent Accuracy: 0.8656195462478184
pearson correaltion 1  (0.2748878145129785, 2.1546903267178903e-11)
pearson correaltion 2 (0.2748878145129785, 2.1546903267178903e-11)
[[  1   0   5   2   0   0   0]
 [  0   0  29  33   3   0   0]
 [  0   3  45  93   8   0   0]
 [  0   0  47 166  22   0   0]
 [  0   0   9  80   9   0   0]
 [  0   0   0  15   1   0   0]
 [  0   0   0   2   0   0   0]]
0.2285871132740509


C:\Users\Ahmed\AppData\Local\Temp\ipykernel_17008\593679167.py:9: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  print("pearson correaltion 2", stats.pearsonr(y_test, pred))


### Sbert with clusters as features with normalized feature vector

In [98]:
essaysNames_train, essaysNames_test, y_train, y_test = train_test_split(listRightNames, y, test_size=0.25, random_state=1000)

essays_train=readTextFromFileName(essaysNames_train)
essays_test =readTextFromFileName(essaysNames_test)


model = SentenceTransformer('all-mpnet-base-v2', device="cuda")
X_train = model.encode(essays_train, show_progress_bar =True, device="cuda")
X_test  = model.encode(essays_test, show_progress_bar =True, device="cuda")

print(X_train.shape)
print(X_test.shape)

X_train_WithClusers = AppendClustersToTfIDf(essaysNames_train,X_train,"sbert")
X_test_WithClusters = AppendClustersToTfIDf(essaysNames_test,X_test,"sbert")

scaler = StandardScaler()
scaler.fit(X_train_WithClusers)
X_TrainedScaled = scaler.fit_transform(X_train_WithClusers)
X_TestScaled = scaler.fit_transform(X_test_WithClusters)

print(X_TrainedScaled.shape)
print(X_TestScaled.shape)

### classification logic #######################

classifier = LogisticRegression() #LogisticRegression()  #DecisionTreeClassifier(max_depth=4)
classifier.fit(X_TrainedScaled, y_train)
score1 = classifier.score(X_TestScaled, y_test)
pred = classifier.predict(X_TestScaled)
print("Accuracy:", score1)
print("ِAdjecent Accuracy:",custom_adjacent_accuracy_score(y_test,pred))
print(confusion_matrix(y_test, pred))

print(cohen_kappa_score(y_test,pred,weights="quadratic"))

Batches:   0%|          | 0/54 [00:00<?, ?it/s]

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

(1716, 768)
(573, 768)
sbert
sbert
(1716, 787)
(573, 787)
Accuracy: 0.3403141361256545
ِAdjecent Accuracy: 0.787085514834206
[[  1   2   4   1   0   0   0]
 [  2   6  29  19   7   1   1]
 [  2  24  51  50  16   6   0]
 [  0  25  55 104  50   1   0]
 [  1   3  21  35  32   6   0]
 [  0   0   2  12   1   1   0]
 [  0   0   0   0   0   2   0]]
0.2685057086165018


C:\Users\Ahmed\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [102]:
classifier = SVC(kernel="linear") #LogisticRegression()  #DecisionTreeClassifier(max_depth=4)
classifier.fit(X_TrainedScaled, y_train)
score1 = classifier.score(X_TestScaled, y_test)
pred = classifier.predict(X_TestScaled)
print("Accuracy:", score1)
print("ِAdjecent Accuracy:",custom_adjacent_accuracy_score(y_test,pred))

print("pearson correaltion 1 ",scipy.stats.pearsonr(y_test, pred))
print("pearson correaltion 2", stats.pearsonr(y_test, pred))

print(confusion_matrix(y_test, pred))

print(cohen_kappa_score(y_test,pred,weights="quadratic"))

Accuracy: 0.3019197207678883
ِAdjecent Accuracy: 0.7539267015706806
pearson correaltion 1  (0.24563573741579806, 2.54447850065474e-09)
pearson correaltion 2 (0.24563573741579806, 2.54447850065474e-09)
[[ 1  2  2  3  0  0  0]
 [ 2 13 32 14  3  1  0]
 [ 2 31 54 41 16  5  0]
 [ 0 43 76 74 39  2  1]
 [ 0 13 22 30 29  4  0]
 [ 0  0  3  9  2  2  0]
 [ 0  0  0  1  1  0  0]]
0.24072711932242263


C:\Users\Ahmed\AppData\Local\Temp\ipykernel_17008\2937301455.py:9: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  print("pearson correaltion 2", stats.pearsonr(y_test, pred))


In [2]:
import fasttext

In [9]:
model = fasttext.train_unsupervised('data/test.txt', model='cbow',minCount=1)

In [ ]:
test = "This is a exmaple of fastText"

model = fasttext.train_supervised('data/test.txt')

In [6]:
print(model.words)
print(model.labels)

['This', 'is', 'a', 'exmaple', 'of', 'fastText']
[]
